# Obteniendo la información para el Scrapping

Importamos las librerias para el scrapping

In [28]:
import pandas as pd
import string
import googlemaps
import numpy as np
import os
import re

## Importando las provincias/distritos a analizar

Primero, leeremos un archivo que contiene los elementos de los ubigeos del INEI. Se puede descargar entrando [aquí](http://webinei.inei.gob.pe:8080/sisconcode/proyecto/index.htm?proyectoTitulo=UBIGEO&proyectoId=3) y dando click a la opción **Excel** en la pestaña *Busqueda por Ubicación Geográfica*

In [14]:
ubigeos = pd.read_excel('ubigeo_inei.xls',skiprows=1)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,01 Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,02 Asunción
4,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,03 Balsas


Necesitamos las provincias y distritos de esta base, así que limpiaremos los valores de estas variables

In [15]:
ubigeos["PROVINCIA"] = ubigeos["PROVINCIA"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["DISTRITO"] = ubigeos["DISTRITO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Asunción
4,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Balsas


Necesitaremos estas dos columnas como listas, pues después las usaremos para crear un DataFrame donde guardaremos los resultados del scrapping

In [16]:
# Creando las listas
provincias = list(ubigeos['PROVINCIA'])
distritos = list(ubigeos['DISTRITO'])

Limpiando las listas

In [17]:
# Eliminando duplicados
provincias = set(provincias)
distritos = set(distritos)

# Eliminando algunos elementos inutiles ('', nan)
provincias = list(filter(lambda x: str(x) != 'nan', provincias))
distritos = list(filter(lambda x: str(x) != 'nan', distritos))

provincias.remove('')
provincias.remove('PROVINCIA')
distritos.remove('')

Eliminando las tildes

In [18]:
# Definimos una función para eliminar las tildes
def normalize(s):
    replacements = (
        ("á", "a"),
        ("à", "a"),
        ("é", "e"),
        ("è", "e"),
        ("í", "i"),
        ("ì", "i"),
        ("ó", "o"),
        ("ò", "o"),
        ("ú", "u"),
        ("ù", "u")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [19]:
# La aplicamos a cada elemento de nuestras provincias/distritos
nro_provs = len(provincias)
for i in range(nro_provs):
    provincias[i] = normalize(provincias[i])
    # De paso, les quitaremos espacios
    provincias[i] = provincias[i].strip()

nro_distrs = len(distritos)
for i in range(nro_distrs):
    distritos[i] = normalize(distritos[i])
    distritos[i] = distritos[i].strip()

In [20]:
print(provincias)
# print(distritos)

['Corongo', 'Huaylas', 'Acobamba', 'Gran Chimu', 'Mariscal Nieto', 'Caraveli', 'Jorge Basadre', 'Azangaro', 'Mariscal Luzuriaga', 'Putumayo', 'Calca', 'Lamas', 'Chanchamayo', 'Tarma', 'Huamalies', 'Sucre', 'Chumbivilcas', 'Puerto Inca', 'Hualgayoc', 'San Pablo', 'Manu', 'Cusco', 'Utcubamba', 'Angaraes', 'Ascope', 'Candarave', 'Bagua', 'Victor Fajardo', 'Tacna', 'Grau', 'Yauli', 'Huaraz', 'Vilcas Huaman', 'Cangallo', 'Huarmey', 'Bellavista', 'Huaral', 'Oxapampa', 'Huallaga', 'Lima', 'Tocache', 'Ilo', 'Bolivar', 'Paucar del Sara Sara', 'Chucuito', 'Pisco', 'Aija', 'Rioja', 'Churcampa', 'Caylloma', 'Huancavelica', 'Huaytara', 'Quispicanchi', 'Luya', 'Pacasmayo', 'Dos de Mayo', 'Oyon', 'Condesuyos', 'Zarumilla', 'Huacaybamba', 'El Dorado', 'Melgar', 'Huanta', 'Huarochiri', 'Huancabamba', 'Daniel Alcides Carrion', 'Acomayo', 'Canas', 'Satipo', 'Asuncion', 'Ucayali', 'Sihuas', 'Maynas', 'La Mar', 'Carabaya', 'Huanca Sancos', 'La Union', 'Parinacochas', 'Purus', 'Talara', 'Espinar', 'Tahuaman

# Scrapping de Distancias y Tiempos de viaje

Para realizar este scrapping, se tomará como referencia esta [guía](https://www.datahubbs.com/google-maps-python/)

### Matriz a nivel de Provincias

Lo primero será crear un DataFrame con los distritos/provincias de origen y de llegada.

In [21]:
first_col = ["Origin Province\Destination Province"]
prov_distances = pd.DataFrame({"Origin Province\Destination Province": provincias}, 
                        columns = first_col+provincias)
prov_distances

,Origin Province\Destination Province,Corongo,Huaylas,Acobamba,Gran Chimu,Mariscal Nieto,Caraveli,Jorge Basadre,Azangaro,Mariscal Luzuriaga,...,Padre Abad,Julcan,Sanchez Carrion,Yauyos,Abancay,Morropon,San Roman,Piura,Mariscal Caceres,Loreto
0,Corongo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Huaylas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acobamba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gran Chimu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mariscal Nieto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Morropon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,San Roman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,Piura,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,Mariscal Caceres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creando listas con los origenes/destinos para reemplazar en el dataframe

In [46]:
# Como queremos iterar sobre los nombres de las provincias
origin = []
destination = []

for prov_row in provincias:
    for prov_col in provincias: 
        origin.append(prov_row+"+"+"Peru")
        destination.append(prov_col+"+"+"Peru")

## Usando el paquete `googlemaps`

Ahora que ya tenemos todos los datos listos, nos falta activar la clase `googlemaps` para que podamos extraer tiempos y distancias. Ello se puede hacer así:

1. Importamos la librería googlemaps (Hecho al inicio de este jupyter)
1. Activamos el API `Distance Matrix API` en Google Cloud Platform (se puede encontrar una guía [aquí](https://www.youtube.com/watch?v=n1UorU1PALk))
1. Creamos una API key para poder usar el item anterior (guía [aquí](https://www.youtube.com/watch?v=2_HZObVbe-g))
1. Activaremos el cobro por el uso del API (Nos daran una prueba gratuita. La guía está [aquí](https://support.google.com/googleapi/answer/6158867?hl=en))

Sin un API key no podrán seguir el proceso, pero es de uso **personal**. Por ello, nunca la debemos escribir directamente en nuestro código; sin embargo, aprenderemos como utilizarla manteniendola privada y lo haremos usando **variables del sistema**

En el Menú de Inicio de Windows, busquen "environment" o "variables". Escojan la opción **"Editar las variables de entorno del sistema"**
![](images\env1.jpg)

Abranlo y entren a la pestaña `Opciones Avanzadas`

![](images\env2.jpg)

Hagan click a la opción `Variables de entornno`
![](images\env3.jpg)

Vayan a la sección `Variables de entorno` y denle click a **Nueva...**
![](images\env4.jpg)

Coloquen el nombre que prefieran (en mi caso, usé  `maps_key`) y, como valor, la API Key correspondiente (no la deben colocar entre comillas)
![](images\env5.jpg)

Denle Aceptar a esta ventana y luego a las anteriores hasta que se cierren todas. Cuando estén cerradas, reinicien su computadora y ya deberían poder usarla

Cuando ya tengan su API Key lista, llamenla de la siguiente manera

In [26]:
key = os.environ.get('maps_key')

Ahora, activaremos la clase google maps usando nuestro key

In [29]:
gmaps = googlemaps.Client(key)

Crearemos ahora nuestra matriz de distancias y tiempos usando esa clase

In [45]:
provs_mat = gmaps.distance_matrix(origins=origin[0], destinations=destination[1], mode='driving')
provs_mat

{'destination_addresses': ['Huaylas 02185, Peru'],
 'origin_addresses': ['Corongo Province, Peru'],
 'rows': [{'elements': [{'distance': {'text': '105 km', 'value': 105439},
     'duration': {'text': '3 hours 31 mins', 'value': 12674},
     'status': 'OK'}]}],
 'status': 'OK'}

**TODO:** Iterar a través de cada elemento de origen y provincia, capturando las distancias y guardandolas en las celdas correspondientes del data frame